In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%cd /content/gdrive/My Drive/Colab Notebooks/2019NN/

/content/gdrive/My Drive/Colab Notebooks/2019NN


In [1]:
!ls

data  data_helpers  Train_Hotel.ipynb  Train_Hotel.py


In [6]:
!curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


# Start

In [1]:
from fastai.text import *
from data_helpers.Data import *
from fastai.text.transform import *

In [3]:
torch.cuda.is_available()

True

In [4]:
torch.cuda.device_count()

1

In [5]:
torch.cuda.get_device_name(device=0)

'Tesla P4'

In [2]:
sent_num_file = ["aspect_0.count", "test_aspect_0.count"]
rating_file = ["aspect_0.rating", "test_aspect_0.rating"]
content_file = ["aspect_0.txt", "test_aspect_0.txt"]

dataset_dir = "./data/hotel_balance_LengthFix1_3000per/"

In [3]:
def concat_to_doc(sent_list, sent_count):
    start_index = 0
    docs = []
    for s in sent_count:
        doc = " xxPERIOD ".join(sent_list[start_index:start_index + s])
        doc = doc + " xxPERIOD "
        docs.append(doc)
        start_index = start_index + s
    return docs

In [4]:
TRAIN_DATA = 0
TEST_DATA = 1

In [5]:
# Load Count
sent_count_train = list(open(dataset_dir + sent_num_file[TRAIN_DATA], "r").readlines())
sent_count_train = [int(s) for s in sent_count_train if (len(s) > 0 and s != "\n")]
print( sent_count_train[0:5] )

# Load Ratings
aspect_rating_train = list(open(dataset_dir + rating_file[TRAIN_DATA], "r").readlines())
aspect_rating_train = [s for s in aspect_rating_train if (len(s) > 0 and s != "\n")]

aspect_rating_train = [s.split(" ") for s in aspect_rating_train]
aspect_rating_train = np.array(aspect_rating_train)[:, 0:-1]
aspect_rating_train = aspect_rating_train.astype(np.int) - 1
aspect_rating_train = pd.DataFrame(aspect_rating_train)
print( aspect_rating_train.head() )

# Load Sents
sents_train = list(open(dataset_dir + content_file[TRAIN_DATA], "r").readlines())
sents_train = [s.strip() for s in sents_train]
print( sents_train[0:5] )

[10, 14, 14, 24, 23]
   0  1  2  3  4  5
0  4  4  4  4  4  4
1  4  3  4  4  4  4
2  1  0  1  1  2  1
3  3  2  3  2  4  3
4  3  2  1  2  2  4
['we had the most wonderful time at the excellence', 'we opted for the excellence club and it was well worht it for the perks', 'roberto puello who was the club manager did an extra special job of making us feel welcomed', 'we took some great pictures that looked like postcards', 'the food was excellent the pizza was close to being our favorite']


In [6]:
docs_train = concat_to_doc(sents_train, sent_count_train)

docs_train = pd.DataFrame(docs_train)

docs_train.head()

,0
0,we had the most wonderful time at the excellen...
1,a truly fantastic place xmas we have just ret...
2,too sick to enjoy the ride to and form the air...
3,"relaxing and fun , but xxPERIOD i rang in the ..."
4,somewhat excellence we just returned from a on...


In [12]:
docs_train.iloc[0][0]

"we had the most wonderful time at the excellence xxPERIOD we opted for the excellence club and it was well worht it for the perks xxPERIOD roberto puello who was the club manager did an extra special job of making us feel welcomed xxPERIOD we took some great pictures that looked like postcards xxPERIOD the food was excellent the pizza was close to being our favorite xxPERIOD the shows were great especially the michael jackson and the 70's  80's night xxPERIOD the french resturant was probably our favorite xxPERIOD we will definately be back for an anniversary trip xxPERIOD we made some friends but had plenty of privcacy as well xxPERIOD the rooms were great and everytime we left and came back it was cleaned , towels restocked and the bar was as well xxPERIOD "

In [7]:
df_train = pd.concat( [aspect_rating_train, docs_train], axis=1, ignore_index=True )
df_train.head()

,0,1,2,3,4,5,6
0,4,4,4,4,4,4,we had the most wonderful time at the excellen...
1,4,3,4,4,4,4,a truly fantastic place xmas we have just ret...
2,1,0,1,1,2,1,too sick to enjoy the ride to and form the air...
3,3,2,3,2,4,3,"relaxing and fun , but xxPERIOD i rang in the ..."
4,3,2,1,2,2,4,somewhat excellence we just returned from a on...


In [14]:
df_train.shape

(11173, 7)

In [8]:
# Load Count
sent_count_test = list(open(dataset_dir + sent_num_file[TEST_DATA], "r").readlines())
sent_count_test = [int(s) for s in sent_count_test if (len(s) > 0 and s != "\n")]
print( sent_count_test[0:5] )

# Load Ratings
aspect_rating_test = list(open(dataset_dir + rating_file[TEST_DATA], "r").readlines())
aspect_rating_test = [s for s in aspect_rating_test if (len(s) > 0 and s != "\n")]

aspect_rating_test = [s.split(" ") for s in aspect_rating_test]
aspect_rating_test = np.array(aspect_rating_test)[:, 0:-1]
aspect_rating_test = aspect_rating_test.astype(np.int) - 1
aspect_rating_test = pd.DataFrame(aspect_rating_test)
print( aspect_rating_test.head() )

# Load Sents
sents_test = list(open(dataset_dir + content_file[TEST_DATA], "r").readlines())
sents_test = [s.strip() for s in sents_test]

# Sents to Doc
docs_test = concat_to_doc(sents_test, sent_count_test)

docs_test = pd.DataFrame(docs_test)
print( docs_test.head() )

[9, 30, 25, 33, 29]
   0  1  2  3  4  5
0  1  0  0  3  1  1
1  2  2  1  2  3  3
2  4  4  4  3  4  4
3  3  2  3  3  3  4
4  3  4  3  4  4  4
                                                   0
0  definitely not a 5 star resort i 'm dumbfounde...
1  facilities need work xxPERIOD we visited excel...
2  excellence was exactly that xxPERIOD my family...
3  great service , nice hotel , mediocre food xxP...
4  very relaxing experience just returned from my...


In [9]:
docs_test.iloc[0][0]

"definitely not a 5 star resort i 'm dumbfounded that this hotel gets good reviews and is so highly rated xxPERIOD it 's decidedly a 3 star property , not 5 stars as indicated xxPERIOD the rooms are very dated and run down , old crappy beds and pillows , an old tv and overall poorly maintained xxPERIOD the whole property is pretty run down and old - looking xxPERIOD the food is subpar , not one meal i had would be called great xxPERIOD the service is uneven and the staff is poorly trained and uninformed xxPERIOD many do not comprehend english xxPERIOD the beach is great , it 's the only redeeming factor xxPERIOD however the resort is a 1- hour taxi trip from the airport xxPERIOD "

In [10]:
df_test = pd.concat( [aspect_rating_test, docs_test], axis=1, ignore_index=True )
df_test.head()

,0,1,2,3,4,5,6
0,1,0,0,3,1,1,definitely not a 5 star resort i 'm dumbfounde...
1,2,2,1,2,3,3,facilities need work xxPERIOD we visited excel...
2,4,4,4,3,4,4,excellence was exactly that xxPERIOD my family...
3,3,2,3,3,3,4,"great service , nice hotel , mediocre food xxP..."
4,3,4,3,4,4,4,very relaxing experience just returned from my...


In [11]:
df_test.shape

(3739, 7)

# Language Model Data Bunch

In [12]:
lmdb = TextLMDataBunch.from_df("./data/", train_df=df_train, valid_df=df_test, text_cols=6, label_cols=[0,1,2,3,4,5])

In [13]:
len( lmdb.vocab.itos )

23008

In [14]:
lmdb.vocab.itos[0:8]

['xxunk', 'xxpad', 'xxbos', 'xxeos', 'xxfld', 'xxmaj', 'xxup', 'xxrep']

In [15]:
lmdb.show_batch()

idx,text
0,"were great especially the michael jackson and the 70 's 80 's night xxperiod the french resturant was probably our favorite xxperiod we will definately be back for an anniversary trip xxperiod we made some friends but had plenty of xxunk as well xxperiod the rooms were great and everytime we left and came back it was cleaned , towels restocked and the bar was as well xxperiod xxbos a"
1,xxperiod if you 're craving the coconut meat inside the coconut ( which was tasty ) then ask the bartender to crack open the coconut after you ' ve finished your drink xxperiod the lobster house was * hot * and if it 's been a hot day i would not recommend eating there xxperiod also we found the steamed lobster to be much better than the grilled ( we
2,"advance to arrange a taxi , driver was waiting with a sign xxperiod xxunk five minutes , sixty bucks and a ten dollar tip later arrived at excellence xxperiod the drive nothing to worry about , xxunk through the potholes , application of the horn as an occasional xxunk aid , all fairly standard in our experience xxperiod you never really get going all that fast xxperiod certainly less of"
3,"check more prices at the vendors nextdoor they will sell it xxperiod those guys are some competition with eachother xxperiod they put towels out each hour , around 3 pm they disappear but you can find them at the clubhouse ( on beach its a mini gift shop / where you meet for the horse ride ) or at the pool behind the bar there is a hut that they"
4,might have been some xxunk with the hotel but that is to be expected as any place ages xxperiod the ride from the airport was about an hour in a mini van which was included in our package xxperiod our driver was great he stopped along the way so the guys could grab a couple of beers xxperiod our check - in was a breeze and the bellboys were very


In [16]:
moms = (0.8,0.7)

In [17]:
learn = language_model_learner(lmdb, AWD_LSTM)
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-2), moms=moms)

epoch,train_loss,valid_loss,accuracy,time
0,4.000441,3.756649,0.287865,01:57
1,3.788871,3.586668,0.304594,02:02
2,3.631943,3.490414,0.315590,01:59
3,3.458348,3.431321,0.322942,01:59
4,3.387737,3.421412,0.324441,02:00


In [24]:
learn = language_model_learner(lmdb, AWD_LSTM)
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-2), moms=moms)

epoch,train_loss,valid_loss,accuracy,time
0,3.961397,3.744603,0.289252,06:04
1,3.766985,3.583916,0.304960,06:13
2,3.601011,3.487880,0.315974,06:14
3,3.475492,3.431211,0.322712,06:15
4,3.398683,3.420531,0.324421,06:15


In [18]:
lmdb.save('lm_databunch_hotel.1115')

In [19]:
learn.save('lm_learner_hotel.1115')

In [25]:
learn.save_encoder('lm_enc_hotel.1115')

# Classifier Data Bunch

In [20]:
clas_db = TextClasDataBunch.from_df("./data/",
                                     train_df=df_train,
                                     valid_df=df_test,
                                     text_cols=6,
                                     label_cols=[0,1,2,3,4,5],
                                     vocab=lmdb.train_ds.vocab,
                                     bs=42)
# TODO: currently only loading the aspect sentiment labels

In [21]:
clas_db.show_batch()

text,target
"xxbos "" an extremely disappointing experience "" where do i beging xxperiod first let me just say that originally i had booked a reservation to stay at this hotel for 9 days instead of the two days that i lasted with my family at this hotel xxperiod the original comfirmed reservation was to have taken place from friday august 15 to sunday august xxunk in may this year i was",
xxbos money well spent first of all let me assure everyone that the gran bahia principe ( punta cana ) is a fabulous resort xxperiod my wife myself have just returned home to the uk from a superb two week vacation staying in the punta cana section of the resort xxperiod weeks prior to our departure reviews on tripadvisor were constantly xxunk giving negative comments about the resort ( mainly,
"xxbos warning to all - read before going xxperiod we went to the bahia from may 4 - 11 xxperiod i felt it important to share my experience with all of you as i did read posts from this site prior to booking xxperiod day 1 friday : arrived at hotel , when you get off the bus , the hotel tags your luggage and they tell you to go",
"xxbos great budget resort with good food , poor service and terrible rooms me and my friend booked this vacation with air canada vacations , we stayed at this hotel as it was the cheapest and we were on a budget xxperiod our luggage was lost so that was not a good start to the trip xxperiod the air canada rep at the hotel is fairly useless , her only",
"xxbos we had a fabulous stay xxperiod we stayed at the edenh from march 8 ? xxperiod our stay was beyond fantastic and our only complaint is that by this time next year we won ? ? be able to afford to stay there again xxperiod we arrived at the hotel at approximately 5:30pm , our check in was smooth with absolutely no problems , the bell boy took our",


In [22]:
x,y = clas_db.one_batch()
y[0:5]

tensor([[0., 0., 2., 2., 2., 2.],
        [2., 4., 3., 4., 4., 1.],
        [0., 0., 4., 2., 4., 0.],
        [2., 4., 0., 3., 3., 0.],
        [3., 3., 3., 4., 4., 4.]])

In [23]:
period_index = clas_db.vocab.stoi["xxperiod"]
period_index

10

In [32]:
x[0].cpu()[0:50] == period_index

tensor([False, False, False, False, False, False, False, False, False, False,
        False,  True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False,  True, False, False, False])

In [33]:
clas_db.vocab.textify(x.cpu()[0,0:50])

'xxbos " an extremely disappointing experience " where do i beging xxperiod first let me just say that originally i had booked a reservation to stay at this hotel for 9 days instead of the two days that i lasted with my family at this hotel xxperiod the original comfirmed'

In [24]:
clas_db.save("cls_databunch_hotel.allaspect.1115")

##  -- load LM Databunch and LM Learner

In [4]:
lmdb = load_data("./data/", "hotel_lm_databunch.1001")

In [5]:
learn = language_model_learner(lmdb, AWD_LSTM)
learn.unfreeze()
learn = learn.load("lang_model_hotel")

freezing:WeightDropout(
  (module): LSTM(400, 1152, batch_first=True)
)
freezing:RNNDropout()
freezing:WeightDropout(
  (module): LSTM(1152, 1152, batch_first=True)
)
freezing:RNNDropout()
freezing:WeightDropout(
  (module): LSTM(1152, 400, batch_first=True)
)
freezing:RNNDropout()


## -- load databunch

In [6]:
clas_db = load_data("./data/", "hotel_cls_databunch.aspect_only")

In [7]:
clas_db.batch_size=2

In [8]:
clas_db.c

5

In [14]:
cls_learn = text_classifier_learner(clas_db, AWD_LSTM)

freezing:Embedding(23008, 400, padding_idx=1)
freezing:EmbeddingDropout(
  (emb): Embedding(23008, 400, padding_idx=1)
)
freezing:WeightDropout(
  (module): LSTM(400, 1152, batch_first=True)
)
freezing:RNNDropout()
freezing:WeightDropout(
  (module): LSTM(1152, 1152, batch_first=True)
)
freezing:RNNDropout()
freezing:WeightDropout(
  (module): LSTM(1152, 400, batch_first=True)
)
freezing:RNNDropout()


## ++ classifier test

In [11]:
cls_learn = text_classifier_learner(clas_db, AWD_LSTM)

In [55]:
requires_grad(cls_learn.model[0].module.rnns[0].module)

False

In [53]:
cls_learn.model

SequentialRNN(
  (0): MultiBatchEncoder(
    (module): AWD_LSTM(
      (encoder): Embedding(23008, 400, padding_idx=1)
      (encoder_dp): EmbeddingDropout(
        (emb): Embedding(23008, 400, padding_idx=1)
      )
      (rnns): ModuleList(
        (0): WeightDropout(
          (module): LSTM(400, 1152, batch_first=True)
        )
        (1): WeightDropout(
          (module): LSTM(1152, 1152, batch_first=True)
        )
        (2): WeightDropout(
          (module): LSTM(1152, 400, batch_first=True)
        )
      )
      (input_dp): RNNDropout()
      (hidden_dps): ModuleList(
        (0): RNNDropout()
        (1): RNNDropout()
        (2): RNNDropout()
      )
    )
  )
  (1): PoolingLinearClassifier(
    (layers): Sequential(
      (0): BatchNorm1d(1200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.4, inplace=False)
      (2): Linear(in_features=1200, out_features=50, bias=True)
      (3): ReLU(inplace=True)
      (4): BatchNorm1d(50, 

# Model Test

In [8]:
encoder = learn.model[0]
encoder.reset()

In [9]:
learn.predict("the rooms are very", n_words=20)

'the rooms are very prodding with this xxperiod we got this but we were excited about going to melia tropical due to the fact'

In [3]:
class PoolingLinearClassifier(Module):
    "Create a linear classifier with pooling."
    def __init__(self, layers:Collection[int], drops:Collection[float]):
        mod_layers = []
        if len(drops) != len(layers)-1: raise ValueError("Number of layers and dropout values do not match.")
        activs = [nn.ReLU(inplace=True)] * (len(layers) - 2) + [None]
        for n_in, n_out, p, actn in zip(layers[:-1], layers[1:], drops, activs):
            mod_layers += bn_drop_lin(n_in, n_out, p=p, actn=actn)
        self.layers = nn.Sequential(*mod_layers)

    def forward(self, input:Tuple[Tensor,Tensor, Tensor])->Tuple[Tensor,Tensor,Tensor]:
        raw_outputs,outputs,mask = input
        x = masked_concat_pool(outputs, mask)
        x = self.layers(x)
        return x, raw_outputs, outputs

In [3]:
class ClsModule(Module):
    "Create a linear classifier with pooling."
    def __init__(self, layers:Collection[int], drops:Collection[float]):
        self.sentiment = torch.nn.Linear(1200, 5)
        self.sentiment_sm = torch.nn.Softmax(dim=1)
        self.aspect = torch.nn.Linear(1200, 5)
        self.aspect_sm = torch.nn.Softmax(dim=1)

    def forward(self, input:Tuple[Tensor,Tensor, Tensor])->Tuple[Tensor,Tensor,Tensor]:
        raw_outputs,outputs,mask = input
        
        # flatten doc length dimension
        doc_enc = outputs.contiguous().view(-1, 400)  # [batch_size * doc_length, embedding400]
        # print("number of sentences in docs:")
        n_sent = torch.sum( input==self.vocab.stoi["xxperiod"] , dim=1)
        # print("locating period marks")
        period_index = input.view(-1)==self.vocab.stoi["xxperiod"]
        # selecting only the encoder output at period marks
        sent_output = doc_enc[period_index, :]  # [total n_sentence, embedding]
        
        return x, raw_outputs, outputs

In [10]:
class ClsNet(torch.nn.Module):
    def __init__(self, encoder, vocab):
        super(ClsNet, self).__init__()
        self.vocab = vocab
        self.enc = encoder
        self.sentiment = torch.nn.Linear(400, 5)
        self.sentiment_sm = torch.nn.Softmax(dim=1)
        self.aspect = torch.nn.Linear(400, 5)
        self.aspect_sm = torch.nn.Softmax(dim=1)
        
    def forward(self, x):
        # encode whole doc
        enc_result = self.enc(x)
        doc_enc = enc_result[0][2]  # [batch_size, doc_length, embedding]
        # print("shape of doc encoder:")
        # print( doc_enc.shape )
        # print("doc_enc grad_fn:")
        # print(doc_enc.grad_fn)
        
        # flatten doc length dimension
        doc_enc = doc_enc.contiguous().view(-1, 400)  # [batch_size * doc_length]
        # print("doc_enc grad_fn:")
        # print(doc_enc.grad_fn)
        
        # print("number of sentences in docs:")
        n_sent = torch.sum( x==self.vocab.stoi["xxperiod"] , dim=1)
        # print(n_sent)
        
        # print("locating period marks")
        period_index = x.view(-1)==self.vocab.stoi["xxperiod"]
        # print(period_index.shape)
        # print(torch.sum(period_index))

        # selecting only the encoder output at period marks
        sent_output = doc_enc[period_index, :]  # [total n_sentence, embedding]
        # print(sent_output)
        # print(sent_output.shape)
        
        sentiment_dist = self.sentiment(sent_output)   # [total n_sentence, embedding]
        sentiment_dist = self.sentiment_sm(sentiment_dist)
        aspect_dist = self.aspect(sent_output)         # [total n_sentence, embedding]
        aspect_dist = self.aspect_sm(aspect_dist)
        # print("sentiment dist weight:")
        # print(sentiment_dist.grad_fn)
        # print("aspect dist weight:")
        # print(aspect_dist.grad_fn)
        
        sent_bmm = torch.bmm(sentiment_dist.unsqueeze(2), aspect_dist.unsqueeze(1))
        # print("sent bmm:")
        # print(sent_bmm.dtype)
        # print(sent_bmm.shape)  # [total n_sentence, sentiment, aspect]
        # print(sent_bmm.grad_fn)
        
        cur = 0
        result = []
        for n_sent in n_sent :
            # print("-----")
            doc = sent_bmm[(cur):(cur+n_sent), :, :]
            # print(doc.shape)
            doc = torch.mean(doc, dim=0, keepdim=True)
            # print(doc.shape)
            result.append(doc)
        
        result = torch.cat( result, dim=0 )
        # print(result.dtype)
        
        return result
        

In [11]:
net = ClsNet(encoder, lmdb.vocab)
net.train()

ClsNet(
  (enc): AWD_LSTM(
    (encoder): Embedding(23008, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(23008, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (sentiment): Linear(in_features=400, out_features=5, bias=True)
  (sentiment_sm): Softmax(dim=1)
  (aspect): Linear(in_features=400, out_features=5, bias=True)
  (aspect_sm): Softmax(dim=1)
)

### // run the net

In [ ]:
x,y = clas_db.one_batch()
input = x[0:2]
label = y[0:2]
label

tensor([[0., 0., 2., 2., 2., 2.],
        [4., 4., 3., 3., 3., 3.]])

In [ ]:
x.shape

torch.Size([48, 2400])

In [ ]:
y.shape

torch.Size([48, 6])

In [ ]:
alabel = label[:,1:6]
alabel = alabel.cuda()
alabel

tensor([[0., 2., 2., 2., 2.],
        [3., 3., 4., 4., 4.]], device='cuda:0')

In [ ]:
result = net(input.cuda())

shape of doc encoder:
torch.Size([2, 2400, 400])
doc_enc grad_fn:
doc_enc grad_fn:
number of sentences in docs:
tensor([80, 79], device='cuda:0')
locating period marks
sentiment dist weight:
aspect dist weight:
sent bmm:
torch.float32
torch.Size([159, 5, 5])
-----
torch.Size([80, 5, 5])
torch.Size([1, 5, 5])
-----
torch.Size([79, 5, 5])
torch.Size([1, 5, 5])
torch.float32


In [ ]:
result.shape

torch.Size([2, 5, 5])

In [ ]:
result

tensor([[[0.0444, 0.0395, 0.0406, 0.0400, 0.0398],
         [0.0419, 0.0373, 0.0383, 0.0377, 0.0375],
         [0.0390, 0.0347, 0.0357, 0.0351, 0.0350],
         [0.0439, 0.0390, 0.0401, 0.0395, 0.0393],
         [0.0482, 0.0428, 0.0441, 0.0434, 0.0431]],

        [[0.0444, 0.0395, 0.0406, 0.0400, 0.0398],
         [0.0419, 0.0373, 0.0383, 0.0378, 0.0375],
         [0.0390, 0.0347, 0.0357, 0.0352, 0.0350],
         [0.0439, 0.0390, 0.0401, 0.0395, 0.0393],
         [0.0482, 0.0428, 0.0440, 0.0434, 0.0431]]], device='cuda:0',
       grad_fn=<CatBackward>)

## define loss

In [14]:
class MultiLabelCEL(nn.CrossEntropyLoss):
    def forward(self, input, target):
        # print("in multi label cel")
#         print(input.shape)
#         print(target.shape)
        i = input.view(-1, 5)    # flatten the aspect dimension, [batch*aspect, sentiment]
        t = target.contiguous().view(-1).long()  # flatten the aspect dimension
#         print(i.shape)
#         print(t.shape)
        loss = super(MultiLabelCEL, self).forward(i, t)
        return loss

In [15]:
l = MultiLabelCEL()

## // test loss

In [39]:
output = l(result, alabel.long() )
output

NameError: ignored

In [ ]:
output.backward()

In [ ]:
net.aspect.weight.grad

tensor([[ 1.5610e-04, -3.1482e-04, -9.7833e-05,  ..., -4.2091e-04,
         -1.0611e-05,  1.2952e-04],
        [ 2.8048e-04, -5.6867e-04, -1.7160e-04,  ..., -7.5715e-04,
         -2.0188e-05,  2.2874e-04],
        [-2.9111e-04,  5.8928e-04,  1.7540e-04,  ...,  7.9670e-04,
          2.0656e-05, -2.3726e-04],
        [-1.3928e-06, -4.8399e-06, -3.8973e-07,  ..., -8.2829e-06,
         -6.2778e-07,  5.3859e-07],
        [-1.4408e-04,  2.9905e-04,  9.4420e-05,  ...,  3.8965e-04,
          1.0771e-05, -1.2154e-04]], device='cuda:0')

In [ ]:
net.aspect.weight.device

device(type='cuda', index=0)

In [61]:
net.enc.reset()

In [20]:
requires_grad(net.enc.rnns[0].module)

False

## training

In [1]:
!nvidia-smi

Thu Oct 10 22:20:42 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.104      Driver Version: 410.104      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    24W /  75W |    699MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [17]:
def model_split(model:nn.Module) -> List[nn.Module]:
    "Split a RNN `model` in groups for differential learning rates."
    groups = [[model.enc.encoder, model.enc.encoder_dp]]
    groups += [[rnn, dp] for rnn, dp in zip(model.enc.rnns, model.enc.hidden_dps)]
    return groups + [[ model.sentiment, model.aspect ]]

In [18]:
my_learner = RNNLearner(data=clas_db,
                     model=net,
                     split_func=model_split,
                     opt_func=torch.optim.Adam,
                     loss_func=l)

In [19]:
my_learner.freeze()

freezing:Embedding(23008, 400, padding_idx=1)
freezing:EmbeddingDropout(
  (emb): Embedding(23008, 400, padding_idx=1)
)
freezing:WeightDropout(
  (module): LSTM(400, 1152, batch_first=True)
)
freezing:RNNDropout()
freezing:WeightDropout(
  (module): LSTM(1152, 1152, batch_first=True)
)
freezing:RNNDropout()
freezing:WeightDropout(
  (module): LSTM(1152, 400, batch_first=True)
)
freezing:RNNDropout()


In [24]:
moms = (0.8, 0.8, 0.8, 0.8, 0.7)

In [1]:
my_learner.fit_one_cycle(4, moms)

NameError: name 'my_learner' is not defined

# TESTS

In [ ]:
a = torch.tensor([ [[1,2],[3,4]], [[5,6],[7,8]] ] )
a

tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])

In [ ]:
a.shape

torch.Size([2, 2, 2])

In [ ]:
a.view(-1,2)

tensor([[1, 2],
        [3, 4],
        [5, 6],
        [7, 8]])

In [ ]:
clas_db.vocab.textify(input.cpu()[0,:])

'xxbos " an extremely disappointing experience " where do i beging xxperiod first let me just say that originally i had booked a reservation to stay at this hotel for 9 days instead of the two days that i lasted with my family at this hotel xxperiod the original comfirmed reservation was to have taken place from friday august 15 to sunday august xxunk in may this year i was considering going back to puerto rico for a longer stay than the 6 days i stood at the san juan marriott resort and stellaris casino in the condado area last year in october xxperiod we had such a great time specially at the marriott that i wrote a beautiful review about the hotel here on tripadvisor and in the review i had stated that if we will ever go back to puerto rico this will be the hotel that we will go back to xxperiod i made a decision back in may this year to at least try another hotel just to see how good other hotels may fair against the marriott xxperiod that is why after reading so many reviews about

In [ ]:
b = torch.tensor([ [1,2], [3,4] ])
b

tensor([[1, 2],
        [3, 4]])

In [ ]:
c = torch.tensor([ [1,1,1], [2,2,2] ])
c

tensor([[1, 1, 1],
        [2, 2, 2]])

In [ ]:
result = torch.bmm(b.unsqueeze(2), c.unsqueeze(1))
result

tensor([[[1, 1, 1],
         [2, 2, 2]],

        [[6, 6, 6],
         [8, 8, 8]]])

In [ ]:
result.shape

torch.Size([2, 2, 3])

In [ ]:
b = torch.tensor( [0,0,1,0,0,0,0,1,0,0,0,1] )
b

tensor([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1])

In [ ]:
ind = b == 1
ind

tensor([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1], dtype=torch.uint8)

In [ ]:
np.add.reduceat(b, [0,3,8])

AttributeError: 'function' object has no attribute 'reduceat'

In [ ]:
torch.cumsum(torch.tensor([79,79]),0)

tensor([ 79, 158])

In [ ]:
b[0:-1]

tensor([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0])